# Agentic retrieval using Azure AI Search and Azure AI Agent Service

Use this notebook to create an agentic retrieval pipeline built on Azure AI Search and an Azure AI Agent.

In this walkthrough, you will:

+ Create an "earth_at_night" search index
+ Load it with documents from a GitHub URL
+ Create a search agent on Azure AI Search that points to an LLM for intelligent query planning
+ Create a Foundry agent in Azure AI Foundry to determine when queries are needed
+ Create a Azure AI Agent tool (client) to orchestrate all requests
+ Start a chat with the agent

This notebook is referenced in [Build an agentic retrieval pipeline in Azure AI Search](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-pipeline).

This exercise differs from the [Agentic Retrieval Quickstart](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval) in how it uses Azure AI Agent to determine whether to retrieve data from the index, and how it uses an agent tool for orchestration.

## Prerequisites

+ Azure AI Search, basic tier or higher, in any region that supports semantic ranker.

+ Azure OpenAI, and you should have an **Azure AI Developer** role assignment to create a Foundry project.

+ An [Azure AI agent and Foundry project](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal), created in the Azure AI Foundry portal, with the basic setup, used for creating the Foundry agent.

+ A deployment of a [supported model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models) in your Foundry project. This notebook uses gpt-4o-mini. We recommend 100,000 token capacity. You can find capacity and the rate limit in the model deployments list in the Azure AI Foundry portal.

## Set up connections

Save the `sample.env` file as `.env` and then modify the environment variables to use your Azure endpoints. You need endpoints for:

+ Azure AI Search
+ Azure OpenAI
+ Azure AI Foundry project

You can find endpoints for Azure AI Search and Azure OpenAI in the [Azure portal](https://portal.azure.com).

You can find the project connection string in the Azure AI Foundry portal:

1. Sign in to the [Azure AI Foundry portal](https://ai.azure.com) and open your project. 

1. In the **Project details** tile, find and copy the **Project connection string**. 

   A hypothetical connection string might look like this: `eastus2.api.azureml.ms;00000000-0000-0000-0000-0000000000;rg-my-resource-group-name;my-foundry-project-name`

1. Check the authentication type for your Azure OpenAI resource and make sure it uses an API key shared to all projects. Still in **Project details**, expand the **Connected resources** tile to view the authentication type for your Azure OpenAI resource.

If you don't have an Azure OpenAI resource in your Foundry project, revisit the model deployment prerequisite. A connection to the resource is created when you deploy a model.


## Load Connections

We recommend creating a virtual environment to run this sample code. In Visual Studio Code, open the control palette (ctrl-shift-p) to create an environment. This notebook was tested on Python 3.10.

Once your environment is created, set the endpoints used to run the retrieval pipeline.

In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
project_conn_str = os.environ["PROJECT_CONNECTION_STRING"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4o-mini")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4o-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4o-mini")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")
api_version = "2025-05-01-Preview"

## Create search index on Azure AI Search

This steps create a search index that contains plain text and vector content. You can use any existing search index, but it must meet the [criteria for agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schmea requirement is that is has a semantic configuration, with a `default_configuration_name`.

In [2]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")


Index 'earth_at_night' created or updated successfully


## Upload sample documents

This sample uses data from NASA's Earth at Night e-book. It's retrieved from the sample data GitHub repository and passed to the search client for indexing.

In [3]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")


Documents uploaded to index 'earth_at_night'


## Create a search agent on Azure AI Search

This steps creates a search agent on Azure AI Search. This agent is a wrapper to a large language model, used for sending queries to an agentic retrieval pipeline. The maximum output size refers to the query response. Setting this value helps you control token usage and how many tokens are sent to the LLM.

In [4]:

from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters
from azure.search.documents.indexes import SearchIndexClient

agent = KnowledgeAgent(
    name=agent_name,
    models=[
        KnowledgeAgentAzureOpenAIModel(
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_gpt_deployment,
                model_name=azure_openai_gpt_model
            )
        )
    ],
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=index_name,
            default_reranker_threshold=2.5
        )
    ],
    request_limits=KnowledgeAgentRequestLimits(
        max_output_size=10000
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_agent(agent)
print(f"Search agent '{agent_name}' created or updated successfully")

Search agent 'earth-search-agent' created or updated successfully


## Create an Azure AI Agent

In the Azure AI Foundry, an agent is a smart micro-service that can do RAG. The purpose of this specific agent is to decide when to send a query to the agentic retrieval pipeline.

In [ ]:
from azure.ai.projects import AIProjectClient

project_client = AIProjectClient.from_connection_string(project_conn_str, credential=credential)

project_client.agents.list_agents()

{'object': 'list', 'data': [{'id': 'asst_sXngVG4fzu0bQJNFmTxzX6h4', 'object': 'assistant', 'created_at': 1746940725, 'name': 'earth-search-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': '\nAn Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer.\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_lzafrxILzTmhdFaMMEVjuhE8', 'object': 'assistant', 'created_at': 1746940688, 'name': 'earth-search-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': '\nAn Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer.\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata'

In [ ]:
from azure.ai.projects import AIProjectClient
import json

project_client = AIProjectClient.from_connection_string(project_conn_str, credential=credential)

instructions = """
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""
agent = project_client.agents.create_agent(
    model=agent_model,
    name=agent_name,
    instructions=instructions
)

print(f"AI agent '{agent_name}' created or updated successfully")

AI agent 'earth-search-agent' created or updated successfully


## Add an agentic retrieval tool to AI Agent

An end-to-end pipeline needs an orchestration mechanism for coordinating calls to the retriever and agent. The pattern described in this notebook uses a [tool](https://learn.microsoft.com/azure/ai-services/agents/how-to/tools/function-calling) for this task. The tool calls the Azure AI Search knowledge retrieval client and the Azure AI agent, and it drives the conversations with the user.

In [7]:
from azure.ai.projects.models import FunctionTool, ToolSet, ListSortOrder

from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams

agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=credential)

thread = project_client.agents.create_thread()
retrieval_results = {}

def agentic_retrieval() -> str:
    """
        Searches a NASA e-book about images of Earth at night and other science related facts.
        The returned string is in a JSON format that contains the reference id.
        Be sure to use the same format in your agent's response
    """
    # Take the last 5 messages in the conversation
    messages = project_client.agents.list_messages(thread.id, limit=5, order=ListSortOrder.DESCENDING)
    # Reverse the order so the most recent message is last
    messages.data.reverse()
    retrieval_result = agent_client.knowledge_retrieval.retrieve(
        retrieval_request=KnowledgeAgentRetrievalRequest(
            messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg.content[0].text)]) for msg in messages.data if msg["role"] != "system"],
            target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
        )
    )

    # Associate the retrieval results with the last message in the conversation
    last_message = messages.data[-1]
    retrieval_results[last_message.id] = retrieval_result

    # Return the grounding response to the agent
    return retrieval_result.response[0].content[0].text

# Link to AI Agent Service function calling tutorial
user_functions = { agentic_retrieval }
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset=toolset)


## Start a chat with the agent

During the chat, you use the standard Azure AI agent tool calling APIs.  We send the message with questions, and the agent decides when to retrieve knowledge from your search index using agentic retrieval.

The remaining cells take a closer look at output and show how to add another turn to the conversation.

In [8]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.list_messages(thread_id=thread.id).get_last_text_message_by_role("assistant").text.value

print("Agent response:", output.replace(".", "\n"))


Agent response: The differences in nighttime brightness between suburban belts and urban cores can be attributed to several factors:

1
 **December Brightening**: Suburban areas tend to have a greater relative increase in brightness during December due to seasonal decorations, festive lighting, and residential activity
 In contrast, urban cores may have higher absolute light levels due to commercial lighting, but the types of light and their arrangement can lead to a less pronounced increase compared to the festive displays in suburban settings
 Urban centers often focus on commercial light sources that may not display the same wide variability in illumination as residential areas do during holidays (Source: ref_id 0)


2
 **Visibility of Phoenix's Nighttime Grid**: The sharp visibility of the Phoenix nighttime street grid from space can be attributed to its planned urban layout, which features a grid pattern of streets and blocks that are well-illuminated
 This grid is a result of urb

## Review retrieval activity and results

Each retrieval response from Azure AI Search includes the unified string (grounding data from search search results), the query plan, and  reference data showing which chunks of source document contributed content to the unified string.

In [9]:

retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))


Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1265,
    "output_tokens": 277
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "suburban belts December brightening urban cores comparison"
    },
    "query_time": "2025-05-13T22:22:45.368Z",
    "count": 0,
    "elapsed_ms": 350
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "Phoenix nighttime street grid visibility from space"
    },
    "query_time": "2025-05-13T22:22:45.682Z",
    "count": 2,
    "elapsed_ms": 304
  },
  {
    "id": 3,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "interstate visibility from space midwestern cities"
    },
    "query_time": "2025-05-13T22:22:45.977Z",
    "count": 2,
    "elapsed_ms": 291
  },
  {
    "id": 4,
    "type": "AzureSearchSemanticRanker",
    

## Continue the conversation...

In [10]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="How do I find lava at night?"
)

run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")

output = project_client.agents.list_messages(thread_id=thread.id).get_last_text_message_by_role("assistant").text.value
print("Agent response:", output.replace(".", "\n"))


Agent response: To find lava at night, you can take advantage of various natural and technological methods:

1
 **Natural Glow**: Active lava flows produce a significant glow, which can be visible from a distance at night
 Under good visibility conditions, the bright red-orange light of flowing lava can be seen against the darker landscape
 Observing this from higher vantage points or from areas with minimal light pollution can enhance visibility


2
 **Satellite Imagery**: During nighttime, satellites equipped with capabilities like the VIIRS (Visible Infrared Imaging Radiometer Suite) can capture images of areas with volcanic activity
 These images show the glow of active lava flows clearly against the background, which highlights volcanic eruptions and their effects
 For example, images of Mount Etna and Kilauea have shown their lava flows brightly illuminated at night (Source: ref_id 1, ref_id 4)


3
 **Infrared Imaging**: Thermal imaging can also assist in identifying hot lava spo

## Review retrieval activity and results


In [11]:
retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))

Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1565,
    "output_tokens": 299
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "how to locate lava flows at night"
    },
    "query_time": "2025-05-13T22:23:07.269Z",
    "count": 6,
    "elapsed_ms": 376
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "best tools for viewing lava at night"
    },
    "query_time": "2025-05-13T22:23:07.575Z",
    "count": 1,
    "elapsed_ms": 293
  },
  {
    "id": 3,
    "type": "AzureSearchQuery",
    "target_index": "earth_at_night",
    "query": {
      "search": "safety tips for exploring lava at night"
    },
    "query_time": "2025-05-13T22:23:07.857Z",
    "count": 0,
    "elapsed_ms": 281
  },
  {
    "id": 4,
    "type": "AzureSearchSemanticRanker",
    "input_tokens": 70920
  }
]
Retrieval results
[
  {

## Clean up objects and resources

If you no longer need the resources, be sure to delete them from your Azure subscription.  You can also delete individual objects to start over.

### Delete the agent

In [12]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_agent(agent_name)
print(f"Search agent '{agent_name}' deleted successfully")

Search agent 'earth-search-agent' deleted successfully


### Delete the index

In [13]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_index(index)
print(f"Index '{index_name}' deleted successfully")

Index 'earth_at_night' deleted successfully
